In [1]:
import numpy as np 
import trimesh
import potpourri3d as pp3d
import torch
import scipy

import matplotlib.pyplot as plt 
from matplotlib import colors, cm

In [8]:
mesh = trimesh.load("mesh.obj",maintain_order=True, process=False)


In [9]:
#calculate L, M
def compute_operators(verts_np, faces_np, eps = 1e-8):
    L = pp3d.cotan_laplacian(verts_np, faces_np, denom_eps=1e-10)

    massvec_np = pp3d.vertex_areas(verts_np, faces_np)
    massvec_np += eps * np.mean(massvec_np)

    return L, massvec_np

In [10]:
verts = mesh.vertices
faces = mesh.faces

L,M = compute_operators(verts, faces)

In [11]:
#diffusion
def diffusion(L, M, u, t=0.01):
    
    L = torch.Tensor(L.todense())
    M = torch.Tensor(M)
    u = torch.Tensor(u)
    mat_dense = L.unsqueeze(0) * t
    mat_dense += torch.diag_embed(M).unsqueeze(0) 
    # Factor the system
    cholesky_factors = torch.linalg.cholesky(mat_dense)

    # Solve the system
    rhs = u * M.unsqueeze(-1) 
    rhsT = torch.transpose(rhs, 0,1)
    sols = torch.cholesky_solve(rhsT.unsqueeze(0), cholesky_factors) 
    

    return sols.squeeze(0)

In [12]:
u = np.zeros(verts.shape[0])
u[0] = 1
u_diffuse =diffusion(L,M,u,t=10)

In [13]:
def getColorMap(v):
    """ Maps each value of a given vector to a colour. Min value gets mapped to blue and 
    max value gets mapped to red
    Input: v: A vector
    """
    
    jet = plt.get_cmap('jet')
    cNorm  = colors.Normalize(v.min(), v.max())
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)
    point_colours = np.zeros((mesh.vertices.shape[0], 3))

    for i in range(v.shape[0]):
        point_colours[i] = scalarMap.to_rgba(v[i])[:3]
    return point_colours

In [14]:

colors_init = getColorMap(u)
colors_new = getColorMap(u_diffuse[:,0])

m1 = trimesh.Trimesh(verts,faces,vertex_colors=colors_init)
m1.export('pre.ply')

m2 = trimesh.Trimesh(verts,faces,vertex_colors=colors_new)
m2.export('diffused10.ply')

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 6890\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nelement face 13776\nproperty list uchar int vertex_indices\nend_header\n_%\x9f\xbd\xe7\xdf2?\x86s\xcd=\x80\x00\x00\xff\xa3?\xb4\xbdU\xf8/?\xc3\x9f\xc1=\xff8\x00\xffx\xd2\xa2\xbd\x18x.? (\xd7=\xffG\x00\xff\xe8\xa3\x8c\xbd\x80\x810?<\xdb\xe3=\xff;\x00\xffD\x14\xb3\xbd\xd5\xaf,?b\x84\xd0=\xffh\x00\xff\x88\xd8\xc0\xbd5~-?T\xe5\xbb=\xffh\x00\xffSB\x90\xbd\xee`,?Z\xf6\xe4=\xffs\x00\xff\xd9\xear\xbd\xf9h-?\x1d\xca\xf0=\xffz\x00\xff\xb1l\x86\xbd\xbe\x9f"?g\xb9\xec=\xff\xc8\x00\xff\xde;j\xbdP\xfc ?\xd5\xaf\xf4=\xff\xd7\x00\xff\x8cfe\xbd\x1d;$?\x197\xf5=\xff\xc4\x00\xff\xcbh\x84\xbdL\x18%?-[\xeb=\xff\xb6\x00\xff\xafB\x8a\xbd)\x96\x1b?Y\xfc\xe6=\xfe\xed\x00\xffL\xc7\x9c\xbd\x99\xbc\x19?\xfb]\xd8=\xf8\xf5\x00\xff\xb8w\x8d\xbd\xc0\xcd\x16?\x88*\xdc=\xeb\xff\x0